### Import the relevant packages

In [2]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from src.train import train_fn
from src.eval import eval_fn
from src.dataset import create_dataloader , train_augmentation , SegmentationDataset , transform
from src.model import UNet
from src.visualize import visualize_over_epochs
from src.metrics import Metrics
from torch.optim import Adam ,lr_scheduler
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
import cv2
import os
import torch
df = pd.read_csv('data/image_label_paths.csv')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pd.options.display.float_format = "{:,.3f}".format

### Prepare the dataset loader

In [3]:
train_dl = create_dataloader(df, train=True,
                                      transform=transform, augment=train_augmentation,
                                      batch_size=4, shuffle=True, drop_last=True)
test_dl = create_dataloader(df, train=False, 
                                     transform=transform, augment=None, 
                                     batch_size=4, shuffle=True, drop_last=True)

### Initialize the model and optimizer 

In [4]:
model = UNet(encoder_name='timm-efficientnet-b0', encoder_weights='imagenet',in_channels=3 , classes=1, activation=None)
model.to(device)
optimizer = Adam(model.parameters(), lr=1e-2)
scheduler = lr_scheduler.ReduceLROnPlateau(
    optimizer,                                 
    factor=0.5 ,                                
    patience=0 , 
    threshold=0.01,                            
    verbose=True , 
    min_lr=1e-5 ,                               
    threshold_mode='abs')

### Training for 25 epochs

In [5]:
best_loss = np.inf
epochs = 50
train_metrics = Metrics()
valid_metrics = Metrics()
for i in range(epochs):
    print(f'Epoch: {i+1}:')
    
    train_epoch_metrics = train_fn(train_dl, model, optimizer , train_metrics)
    print('\n')
    valid_epoch_metrics = eval_fn(test_dl, model , valid_metrics)
    scheduler.step(valid_epoch_metrics['loss'])
    print('\n')
    epoch_df = pd.DataFrame([train_epoch_metrics, valid_epoch_metrics] , index=['Train' , 'Valid'])
    epoch_df.style.set_properties(**{'text-align': 'left'})
    with pd.option_context('expand_frame_repr', False):
        print (epoch_df)
    print('\n')
    if valid_epoch_metrics['loss'] < best_loss:
        best_loss = valid_epoch_metrics['loss']
        torch.save(model.state_dict(), f'models/best_model.pth')
        print('Model Saved')
    print("\n")
    print('============================================================')

Epoch: 1:
Training ...


100%|██████████| 242/242 [05:35<00:00,  1.39s/it]


Training Done!
Epoch Metrics Saved!
Metrics Reseted!


Validating ...


100%|██████████| 50/50 [01:15<00:00,  1.51s/it]


Validation Done!
Epoch Metrics Saved!
Metrics Reseted!


       loss  accuracy  precision  recall  f1_score  specificity   iou
Train 0.702     0.961      0.537   0.513     0.512        0.982 0.354
Valid 0.675     0.970      0.509   0.555     0.523        0.983 0.361


Model Saved


Epoch: 2:
Training ...


...

Training Done!
Epoch Metrics Saved!
Metrics Reseted!


Validating ...


100%|██████████| 50/50 [00:12<00:00,  3.95it/s]


Validation Done!
Epoch Metrics Saved!
Metrics Reseted!


       loss  accuracy  precision  recall  f1_score  specificity   iou
Train 0.504     0.975      0.709   0.658     0.678        0.989 0.518
Valid 0.463     0.983      0.744   0.657     0.693        0.993 0.536




Epoch: 25:
Training ...


 52%|█████▏    | 126/242 [01:01<00:56,  2.04it/s]


KeyboardInterrupt: 